### **Imports**

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install faster_whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
import torch
import torchaudio
import librosa
import numpy as np

from datasets import Dataset
from collections import Counter
from transformers import pipeline

from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
from faster_whisper import WhisperModel

# Load ASR model
model = WhisperModel("medium")

# Transcribe audio
segments, info = model.transcribe("./b1.mp3", word_timestamps=True)

# Extract words and timestamps
word_timestamps = []
for segment in segments:
    for word in segment.words:
        word_timestamps.append((word.word, word.start, word.end))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

In [ ]:
word_timestamps

[(' I', 0.0, 0.54),
 (" don't", 0.54, 0.78),
 (' oppose', 0.78, 1.12),
 (' war', 1.12, 1.56),
 (' in', 1.56, 2.02),
 (' all', 2.02, 2.32),
 (' circumstances', 2.32, 2.9),
 (' and', 2.9, 3.08),
 (' when', 3.08, 3.18),
 (' I', 3.18, 3.34),
 (' look', 3.34, 3.52),
 (' out', 3.52, 3.72),
 (' over', 3.72, 3.94),
 (' this', 3.94, 4.16),
 (' crowd', 4.16, 4.4),
 (' today,', 4.4, 4.82),
 (' I', 5.4, 5.52),
 (' know', 5.52, 5.7),
 (' there', 5.7, 5.88),
 (' is', 5.88, 6.08),
 (' no', 6.08, 6.3),
 (' shortage', 6.3, 6.76),
 (' of', 6.76, 7.14),
 (' patriots', 7.14, 7.82),
 (' or', 7.82, 8.52),
 (' patriotism.', 8.52, 9.32),
 (' What', 10.04, 10.12),
 (' I', 10.12, 10.3),
 (' do', 10.3, 10.5),
 (' oppose', 10.5, 11.06),
 (' is', 11.06, 11.76),
 (' a', 11.76, 11.92),
 (' dumb', 11.92, 12.08),
 (' war.', 12.08, 12.46)]

### **Constants**

In [ ]:
AUDIO_FILE = "./harvard.wav"

SAMPLING_RATE = 20000
CHUNK_DURATION = 10 # in seconds
CHUNK_OVERLAP_DURATION = 2 # in seconds

EMOTION_TOP_N_RESULTS = 3

### **Preprocessing**

In [ ]:
def create_chunks_with_padding(audio, sr, start, end, chunk_duration, step_size, min_size):

    local_chunks = []
    for i in range(start, end, int(step_size * sr)):
        chunk_end = i + int(chunk_duration * sr)
        chunk = audio[i:chunk_end]

        if len(chunk) < min_size:
            chunk = np.pad(chunk, (0, min_size - len(chunk)), mode="constant")

        local_chunks.append((chunk, i / sr, chunk_end / sr))
    return local_chunks

def chunk_audio_parallel_with_padding(audio_path, chunk_duration=CHUNK_DURATION, overlap=CHUNK_OVERLAP_DURATION, sample_rate=SAMPLING_RATE, num_workers=4):

    audio, sr = librosa.load(audio_path, sr=sample_rate, mono=True)
    step_size = chunk_duration - overlap
    min_size = int(chunk_duration * sr)

    # Divide the audio range into sections for parallel processing
    total_length = len(audio)
    section_size = total_length // num_workers
    sections = [(audio, sr, i, min(i + section_size, total_length), chunk_duration, step_size, min_size)
                for i in range(0, total_length, section_size)]

    # Process each section in parallel
    chunks = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(lambda args: create_chunks_with_padding(*args), sections)
        for result in results:
            chunks.extend(result)

    return chunks

### **Tone / Emotion Analysis**

In [ ]:
pipe = pipeline(
    "audio-classification",
    model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition",
    device=0
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the che

In [ ]:
def process_chunk(chunk_data):
    chunk, start, end = chunk_data
    chunk_results = pipe(chunk)

    print("Debug : ", chunk_results, "\n\n")

    return {
        "timestep": f"{start:.2f}-{end:.2f}",
        "emotion": chunk_results[0]["label"],
        "score": chunk_results[0]["score"]
    }

def analyze_chunks_parallel(chunks, max_workers=8):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_chunk, chunks))
    return results

### **Pitch and Modulation Analysis**

In [ ]:
def extract_pitch(audio, sr):
    pitches, magnitudes = librosa.core.piptrack(y=audio, sr=sr)
    pitch = [np.max(pitches[:, t]) for t in range(pitches.shape[1])]
    return np.array(pitch)

def extract_loudness(audio):
    rms = librosa.feature.rms(y=audio)
    return rms[0]

def analyze_modulation_and_pitch(audio, sr):
    pitch = extract_pitch(audio, sr)
    loudness = extract_loudness(audio)

    pitch_variation = np.std(pitch)
    loudness_variation = np.std(loudness)

    return pitch, loudness, pitch_variation, loudness_variation

### **Feedback Generation**

In [ ]:
# def generate_feedback(pitch, loudness, pitch_variation, loudness_variation, emotion):
#     feedback = {}

#     if emotion == 'happy':
#         feedback["modulation"] = "Great job! Your speech has excellent variation in volume and pitch. It conveys positivity and energy."
#     elif emotion == 'anger':
#         feedback["modulation"] = "Your speech has strong variation in pitch and volume, conveying intensity. Be mindful of the tone to avoid sounding overly aggressive."
#     elif emotion == 'sad':
#         feedback["modulation"] = "Your speech lacks variation, which is typical for a sad tone. Try to vary your pitch and loudness to convey more nuance."
#     elif emotion == 'neutral':
#         feedback["modulation"] = "Your speech is steady, which is good for neutral delivery, but varying pitch and loudness could make it more engaging."

#     if emotion == 'happy':
#         if np.mean(pitch) < 180:
#             feedback["pitch"] = "Your pitch could be a little higher to reflect more excitement. Try to increase your pitch to convey more energy."
#         else:
#             feedback["pitch"] = "Your pitch is in a great range for happy speech. Keep the energy high!"
#     elif emotion == 'anger':
#         if np.mean(pitch) < 220:
#             feedback["pitch"] = "Your pitch is on the lower side for an angry tone. Consider raising your pitch for more intensity and to emphasize anger."
#         else:
#             feedback["pitch"] = "Your pitch is strong and conveys anger well, but be careful not to sound too harsh."
#     elif emotion == 'sad':
#         if np.mean(pitch) > 150:
#             feedback["pitch"] = "For a sad tone, your pitch is higher than expected. Try lowering your pitch slightly to better convey sorrow."
#         else:
#             feedback["pitch"] = "Your pitch matches the sad tone well. Keep it steady and low to maintain the emotional depth."
#     elif emotion == 'neutral':
#         feedback["pitch"] = "Your pitch is within a comfortable range. It's neutral, but adding more variation could enhance engagement."

#     return feedback


def generate_feedback(pitch, loudness, pitch_variation, loudness_variation, emotion):
    feedback = {}

    # Emotion-specific Modulation Feedback
    if emotion == 'happy':
        feedback["modulation"] = (
            "Your speech has great variation in pitch and volume, reflecting positivity and energy. "
            "It engages the listener effectively. Keep maintaining this liveliness!"
        )
    elif emotion == 'anger':
        feedback["modulation"] = (
            "Your speech has strong variation in pitch and volume, conveying intensity. "
            "While it effectively emphasizes anger, be cautious of not sounding too harsh or overwhelming."
        )
    elif emotion == 'sad':
        feedback["modulation"] = (
            "Your speech is relatively flat, which is typical for a sad tone. "
            "Consider varying your pitch and volume slightly to add more emotional nuance while maintaining the somber feel."
        )
    elif emotion == 'neutral':
        feedback["modulation"] = (
            "Your speech is steady and appropriate for neutral delivery. "
            "However, adding more variation in pitch and volume could make your delivery more engaging."
        )
    elif emotion == 'surprise':
        feedback["modulation"] = (
            "Your speech shows dynamic variation, well-suited for a tone of surprise. "
            "Try to keep your pitch and volume shifts balanced to maintain clarity."
        )
    elif emotion == 'disgust':
        feedback["modulation"] = (
            "Your speech conveys disgust effectively with controlled variation. "
            "Ensure your tone doesn't become too monotonous or overly exaggerated."
        )
    elif emotion == 'fear':
        feedback["modulation"] = (
            "Your speech reflects fear with noticeable variation in pitch and volume. "
            "Maintaining a steady tone can help convey controlled anxiety if that's the goal."
        )

    # Pitch Feedback
    mean_pitch = np.mean(pitch)
    if emotion == 'happy':
        if mean_pitch < 180:
            feedback["pitch"] = (
                "Your pitch could be slightly higher to match the excitement associated with happiness. "
                "Raising it will add more enthusiasm to your delivery."
            )
        else:
            feedback["pitch"] = (
                "Your pitch is excellent for a happy tone. It conveys energy and positivity. Great job!"
            )
    elif emotion == 'anger':
        if mean_pitch < 220:
            feedback["pitch"] = (
                "For anger, your pitch is slightly low. Consider raising it to add intensity and emphasize your message."
            )
        else:
            feedback["pitch"] = (
                "Your pitch effectively conveys anger. Ensure it doesn’t become overly sharp to maintain clarity."
            )
    elif emotion == 'sad':
        if mean_pitch > 150:
            feedback["pitch"] = (
                "For a sad tone, your pitch is a bit high. Lowering it slightly can help convey a more somber feel."
            )
        else:
            feedback["pitch"] = (
                "Your pitch aligns well with the sad tone. It's low and steady, which matches the emotion perfectly."
            )
    elif emotion == 'neutral':
        feedback["pitch"] = (
            "Your pitch is within a neutral range, which is appropriate. Adding slight variations could make it more engaging."
        )

    # Loudness Feedback
    mean_loudness = np.mean(loudness)
    if mean_loudness < 0.01:
        feedback["loudness"] = (
            "Your volume is too low, making it hard for the audience to hear clearly. "
            "Increase your volume to ensure your message is heard."
        )
    elif mean_loudness > 0.3:
        feedback["loudness"] = (
            "Your volume is quite high, which may come across as overwhelming. "
            "Lowering it slightly will make your delivery more comfortable for listeners."
        )
    else:
        feedback["loudness"] = (
            "Your volume is well-balanced and appropriate for your tone. Keep up the good work!"
        )

    # Modulation (Pitch & Loudness Variation) Feedback
    if pitch_variation < 20:
        feedback["pitch_variation"] = (
            "Your pitch variation is quite low, which could make your speech sound monotonous. "
            "Try adding more variation to keep your audience engaged."
        )
    elif pitch_variation > 100:
        feedback["pitch_variation"] = (
            "Your pitch variation is too high, making your speech sound inconsistent. "
            "Balancing it can make your delivery smoother."
        )
    else:
        feedback["pitch_variation"] = (
            "Your pitch variation is excellent. It adds dynamism to your speech and keeps it interesting."
        )

    if loudness_variation < 0.01:
        feedback["loudness_variation"] = (
            "Your loudness variation is quite low, making your speech sound flat. "
            "Consider modulating your volume to emphasize key points."
        )
    elif loudness_variation > 0.1:
        feedback["loudness_variation"] = (
            "Your loudness variation is high, which adds energy to your speech. "
            "Ensure it remains controlled to maintain clarity."
        )
    else:
        feedback["loudness_variation"] = (
            "Your loudness variation is well-balanced. It keeps the audience attentive without being overwhelming."
        )

    return feedback


### **Speech Analyzer**

In [ ]:
def classify_audio(examples):
    audio_list = [np.array(audio, dtype=np.float32) for audio in examples["audio"]]

    # Classify emotions for each chunk of audio
    results = pipe(audio_list)

    emotions, scores = [], []

    for res in results :
      t1, t2 = [], []
      for i in range(EMOTION_TOP_N_RESULTS) :
        t1.append(res[i]["label"])
        t2.append(res[i]["score"])
      emotions.append(t1)
      scores.append(t2)

    # emotions = [res[0]["label"] for res in results]
    # scores = [res[0]["score"] for res in results]

    # Initialize the results
    feedback_list = []

    # Process each audio chunk
    for i, audio in enumerate(audio_list):
        # Extract pitch and loudness
        pitch, loudness, pitch_variation, loudness_variation = analyze_modulation_and_pitch(audio, SAMPLING_RATE)

        feedback = generate_feedback(pitch, loudness, pitch_variation, loudness_variation, emotions[i])

        print(feedback)
        # feedback_list.append({
        #     "timestep": examples["timestep"][i],
        #     "emotion": emotions[i],
        #     "score": scores[i],
        #     "pitch": pitch.tolist(),
        #     "loudness": loudness.tolist(),
        #     ...
        #     "feedback": feedback
        # })

    return {"results": feedback_list}

In [ ]:
chunks = chunk_audio_parallel_with_padding(AUDIO_FILE, chunk_duration=CHUNK_DURATION, overlap=CHUNK_OVERLAP_DURATION)
data = [
    {"audio": np.array(chunk[0], dtype=np.float32), "timestep": chunk[1]}
    for chunk in chunks
]
dataset = Dataset.from_list(data)

In [ ]:
# Load the first chunk
first_chunk_audio = chunks[0][0]
first_chunk_start = chunks[0][1]
first_chunk_end = chunks[0][2]

# Analyze modulation and pitch
pitch, loudness, pitch_variation, loudness_variation = analyze_modulation_and_pitch(first_chunk_audio, SAMPLING_RATE)

# Classify the emotion for the first chunk (dummy example: assuming 'happy' as a placeholder)
emotion = 'happy'  # Replace with actual classification if needed

# Generate feedback
feedback = generate_feedback(pitch, loudness, pitch_variation, loudness_variation, emotion)

# Print results
print(f"Chunk Time: {first_chunk_start:.2f}s - {first_chunk_end:.2f}s")
print("Feedback:", feedback)


Chunk Time: 0.00s - 10.00s
Feedback: {'modulation': 'Your speech has great variation in pitch and volume, reflecting positivity and energy. It engages the listener effectively. Keep maintaining this liveliness!', 'pitch': 'Your pitch is excellent for a happy tone. It conveys energy and positivity. Great job!', 'loudness': 'Your volume is well-balanced and appropriate for your tone. Keep up the good work!', 'pitch_variation': 'Your pitch variation is too high, making your speech sound inconsistent. Balancing it can make your delivery smoother.', 'loudness_variation': 'Your loudness variation is well-balanced. It keeps the audience attentive without being overwhelming.'}


In [ ]:
dataset = dataset.map(classify_audio, batched=True, batch_size=8)

### **Just an idea : (Smoothing and N-Gram Like Analysis)**

- Instead of taking the top emotion, take the top 2 or 3
- For every 2-3 timesteps (depending on the `CHUNK_DURATION`), <br>take the 2 most occurring emotion and assign to all of them (**smoothing basically**)
- Using this combination on emotions, make a judgement

In [ ]:
results = dataset.to_pandas()["results"].to_list()

In [ ]:
emotions = [res["emotion"] for res in results]

In [ ]:
import numpy as np
from collections import Counter

SMOOTHING_FACTOR = 3

smoothed_emotions = []

for i in range(0, len(emotions), SMOOTHING_FACTOR):

    temp = emotions[i : i + SMOOTHING_FACTOR]

    flat_temp = [emotion.tolist() if isinstance(emotion, np.ndarray) else emotion for emotion in temp]

    flat_temp = [item for sublist in flat_temp for item in (sublist if isinstance(sublist, list) else [sublist])]

    emotion_counts = Counter(flat_temp)

    most_frequent_emotion = emotion_counts.most_common(2)
    smoothed_emotions.append(most_frequent_emotion)

    print(i, flat_temp, "Most frequent:", most_frequent_emotion)

print("Smoothed Emotions:", smoothed_emotions)

0 ['calm', 'disgust', 'happy', 'disgust', 'calm', 'neutral', 'disgust', 'neutral', 'angry'] Most frequent: [('disgust', 3), ('calm', 2)]
3 ['disgust', 'happy', 'surprised'] Most frequent: [('disgust', 1), ('happy', 1)]
Smoothed Emotions: [[('disgust', 3), ('calm', 2)], [('disgust', 1), ('happy', 1)]]


In [ ]:
import numpy as np
from collections import Counter

def calculate_speaking_score(
    smoothed_video_emotions, smoothed_audio_emotions,
    pitch_variation, loudness_variation,
    words_per_minute, silence_durations,
    average_volume,
    ideal_pitch_range=(20, 100),
    ideal_loudness_range=(0.01, 0.3),
    ideal_wpm_range=(120, 160),
    ideal_volume_range=(60, 80)
):
    score = 100

    # 1. Emotion Mismatch
    video_set = {emotion[0] for smooth_emotion in smoothed_video_emotions for emotion in smooth_emotion}
    audio_set = {emotion[0] for smooth_emotion in smoothed_audio_emotions for emotion in smooth_emotion}

    matches = video_set & audio_set  # Intersection of emotions
    if matches:
        score += len(matches) * 2  # Reward for each match
        print("mismatch emo", score)
    else:
        score -= 5  # Small penalty for no match

    # Cap the emotion score impact between 95 and 100
    score = max(95, min(100, score))
    print("emotion score impact", score)

    # 2. Modulation (Pitch & Loudness)
    if not (ideal_pitch_range[0] <= pitch_variation <= ideal_pitch_range[1]):
        score -= 10  # Penalize unnatural pitch variation
    if not (ideal_loudness_range[0] <= loudness_variation <= ideal_loudness_range[1]):
        score -= 10  # Penalize unnatural loudness variation

    print("pitch", score)

    # 3. Speaking Speed
    if not (ideal_wpm_range[0] <= words_per_minute <= ideal_wpm_range[1]):
        score -= 10  # Penalize for too fast or too slow speech
    silence_penalty = sum([2 if s > 2 else 0 for s in silence_durations])
    score -= min(10, silence_penalty)  # Limit silence penalty impact
    print("speed", score)

    # 4. Volume
    if not (ideal_volume_range[0] <= average_volume <= ideal_volume_range[1]):
        score -= 10  # Penalize for too high or low volume
    print("volume", score)

    # Ensure the score remains within 0–100
    score = max(0, min(100, score))
    return score

# Example Usage:
smoothed_video_emotions = [[('disgust', 3), ('calm', 2)], [('disgust', 1), ('happy', 1)]]
smoothed_audio_emotions = [[('calm', 3), ('neutral', 2)], [('happy', 2), ('angry', 1)]]

pitch_variation = 50  # Std dev of pitch
loudness_variation = 0.02  # Std dev of loudness
words_per_minute = 150  # Number of words spoken per minute
silence_durations = [0.5, 1.0, 3.0, 0.8]  # Silence durations in seconds
average_volume = 65  # Average volume in dB

score = calculate_speaking_score(
    smoothed_video_emotions, smoothed_audio_emotions,
    pitch_variation, loudness_variation,
    words_per_minute, silence_durations,
    average_volume
)

print(f"Speaking Score: {score}")


mismatch emo 104
emotion score impact 100
pitch 100
speed 98
volume 98
Speaking Score: 98
